# Introduction to Data Preparation
This notebook focuses on loading the Airbnb listings dataset, cleaning it, and preparing it for feature engineering. The main steps include:
- Loading data into a Pandas DataFrame
- Randomly sampling 100 rows for live data
- Storing the remainder of the data in a PostgreSQL database
- Removing irrelevant columns
- Handling missing values
- Converting categorical variables

## Part 1: Data Loading and Sampling
### Load Data
First, we load the Airbnb listings dataset into a Pandas DataFrame.



I am working on scenario1: You and a group of friends are considering purchasing a property in Chicago that you can use as an investment. You have heard from other people that they have made a lot of money by renting out either a room or an entire unit (apartment or house). Your friends ask you to analyze data so that they can understand how much you would charge per night based on the type of dwelling you were to purchase.

In [1]:
import pandas as pd
import numpy as np

Load data into a Pandas DataFrame


In [2]:
df = pd.read_csv("listings.csv")

Loading data into a Pandas dataframe randomly selecting 100 rows of the data. Once selected, I saved these rows to new csv AND removed them from primary dataframe. 

In [3]:
sampled_df = df.sample(n=100, random_state=42)

Save the sampled rows to a new CSV file

In [4]:
sampled_df.to_csv("live_data.csv", index=False)

Remove these rows from the original DataFrame


In [5]:
df = df.drop(sampled_df.index)


## Part 2: PostgreSQL Database Connection and Data Storage
### Connect to PostgreSQL Database
Next, we connect to a PostgreSQL database and store the remainder of the data in the raw schema.



In [6]:
from sqlalchemy import create_engine
import psycopg2

Define database connection parameters


In [ ]:
host = '127.0.0.1'
db = 'project_airbnb'
user = 'postgres'
pw = 'your_pass'
port = '5432'

Create a database connection engine


In [8]:
db_conn = create_engine(f"postgresql://{user}:{pw}@{host}:{port}/{db}")

Store the remainder of data in the raw schema


In [9]:
df.to_sql('remainder_data', db_conn, schema='raw', if_exists='replace', index=False)

297

Data Definition_

Field name and description:
•	id - The id of the airbnb
•	name - The name of the airbnb
•	host_id - The id of the host
•	host_name - The name of the host
•	neighborhood - Which area is the airbnb belongs to
•	latitude - The latitude of the position
•	longitude - The longitude of the position
•	room_type - Entire home/apt, Private room or Other
•	price - The price of the apartment (per day)
•	minimum_nights - The least nights you need to book
•	number_of_reviews - The total number of the reviews on this dataset
•	last_review - The last review time
•	reviews_per_month - How many reviews the airbnb can receive per month
•	calculated_host_listings_count - The total listing number of the host
•	availability_365 - The available days

Type of data (nominal, ordinal, discrete, continuous):
o	Nominal: Categories without order (e.g., Neighborhood, name).
o	Discrete: Countable numbers (e.g., price, minimum_nights).
o	Continuous: Measured values (e.g., reviews_per_month).



Analytical Question_

A well-formed question as a complete sentence:
How do different factors (e.g., room_type, number_of_reviews, and neighbourhood) influence the listing’s price?

Identify the target variable for your analysis:
Price 



now let's cleanup and prepare the remaining data for machine learning: handling missing values or outliers, converting categorical variables, etc.

Remove irrelevant columns


In [10]:
columns_to_drop = ['id', 'host_id', 'host_name', 'calculated_host_listings_count', 'last_review']
df = df.drop(columns=columns_to_drop)

Save the cleaned dataset


In [11]:
df.to_csv("cleaned_listings.csv", index=False)

Check for missing values


In [12]:
print(df.isnull().sum())

name                      0
neighbourhood_group    6297
neighbourhood             0
latitude                  0
longitude                 0
room_type                 0
price                     0
minimum_nights            0
number_of_reviews         0
reviews_per_month      1114
availability_365          0
dtype: int64


Visualize outliers using IQR method for numerical columns


In [15]:
numerical_cols = ['price', 'minimum_nights', 'number_of_reviews', 'availability_365']
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    print(f"Outliers for {col}:")
    print(df[(df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR))])

Outliers for price:
                                                   name  neighbourhood_group  \
40             Family Style Home-walk to Aragon/Riviera                  NaN   
62                      Chicago Lakefront Sailing Yacht                  NaN   
64                  Drop-dead fab on the Lake  2bd/2bth                  NaN   
88                   Italian style in elegant apartment                  NaN   
123                     Lincoln Park Single Family Home                  NaN   
...                                                 ...                  ...   
6307  Gorgeous NEW 5 BDR/ 4BA in Logan Square w/Park...                  NaN   
6378   Sundrenched Northeast Corner Unit in South Loop.                  NaN   
6383        Luxury condo in Lincoln Park prime location                  NaN   
6391                 Space for Professional Events Only                  NaN   
6394  Two story Penthouse w private Rooftop & Great ...                  NaN   

        neighbourho

Convert categorical variables


In [16]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['room_type'] = le.fit_transform(df['room_type'])
df['neighbourhood'] = le.fit_transform(df['neighbourhood'])

Store cleaned data in the cleaned schema


In [17]:
df.to_sql('cleaned_data', db_conn, schema='cleaned', if_exists='replace', index=False)

297